In [1]:
# Set up

# path = "./day6_input.txt"
path = "./day6_test.txt"

with open(path) as file:
    lines = file.read().splitlines()

grid = [[s for s in line] for line in lines]   # this line might not be necessary as the string is already an array of strings

start_pos = [(r,c) for r in range(len(grid)) for c in range(len(grid[0])) if grid[r][c] == '^'][0] # [0] extracts the tuple from the list of 1

print(start_pos)

sum = 0 
for r in grid:
    sum += r.count('#')
print(f"There are {sum} #")
print(f"there are : {len(grid)} rows")
print(f"There are : {len(grid[0])} columns")

(6, 4)
There are 8 #
there are : 10 rows
There are : 10 columns


In [ ]:
import copy
import os

class GridStateMachine:

    UP = (-1,0)
    RIGHT = (0,1)
    DOWN = (1,0)
    LEFT = (0,-1)
    
    DIRECTIONS = (UP,RIGHT,DOWN,LEFT)

    def __init__(self, grid, start, dir_ind, with_obs:bool=False):
        self.grid = grid
        self.pos = start
        self.with_obs = with_obs
        self.dir_ind = dir_ind
        self.state_list = [(self.pos, self.dir_ind)]
        self.obs_list = []


    def run(self) -> bool: # return true if a loop, false if exits grid
        run = True
        while run: 
            # get next_pos by addind direction to pos 
            next_pos = self.next_pos()
            
            # check for loop with obstacle in next_pos
            if self.with_obs: self.check_with_obs(next_pos)
                
            # out of bounds
            if self.out_of_bounds(next_pos): 
                return False
            # obstacle
            elif self.grid[next_pos[0]][next_pos[1]] in ['#','O']: 
                self.turn()
            # update pos
            else: 
                self.pos = next_pos
                self.mark_pos()
                # check if state has been visited before, if so return True (ie there is a loop)
                loop = self.check_state()
                if loop: 
                    self.write_grid_to_file()
                    return True


    def write_grid_to_file(self):

        # find the O
        pos = (0,0)
        for r_ind, r in enumerate(self.grid):
            for c_ind, c in enumerate(r):
                if c == 'O':
                    pos = (r_ind, c_ind)
        if os.path.exists("./outputs"):
            os.rmdir("./outputs")
        os.mkdir("./outputs")
        with open(f"./outputs/grid_for_obs_at_{pos}.txt", "w") as file:
            for row in self.grid:
                file.write("".join(map(str, row)) + "\n")

    def check_with_obs(self, next_pos):
        if next_pos == (7,6):
            pass
        # take a copy of the grid
        grid_copy = copy.deepcopy(self.grid)
        # add the obstacle at next_pos
        if not self.out_of_bounds(next_pos):
            grid_copy[next_pos[0]][next_pos[1]] = 'O'
            gsm = GridStateMachine(grid_copy, self.pos, self.dir_ind, False)
            loop_found = gsm.run()
            if loop_found: 
                print("loop_found with obstacle position: ", next_pos)
                if not next_pos in self.obs_list: self.obs_list.append(next_pos)
            
    # Returns True if state has already been visited ie is a loop, if not adds state to state_list and returns false
    def check_state(self):
        state = (self.pos, self.dir_ind)
        if state in self.state_list:
            return True
        else:
            self.state_list.append(state)
            return False
    
    def mark_pos(self):
        current_marker = self.grid[self.pos[0]][self.pos[1]]
        if current_marker == '.':
            if self.dir_ind in [0,2]: 
                self.grid[self.pos[0]][self.pos[1]] = '|'
            else: 
                self.grid[self.pos[0]][self.pos[1]] = '-'
        elif current_marker == '|' and self.dir_ind in [1,3]:
            self.grid[self.pos[0]][self.pos[1]] = '+'
        elif current_marker == '|' and self.dir_ind in [0,2]: 
            pass
        elif current_marker == '-' and self.dir_ind in [0,2]: 
            self.grid[self.pos[0]][self.pos[1]] = '+'
        elif current_marker == '-' and self.dir_ind in [1,3]:
            pass
        elif current_marker in ['^','+']:
            pass
        else:
            raise Exception('marker error with marker: ', current_marker)

    def next_pos(self):
        dir = self.DIRECTIONS[self.dir_ind]
        return (self.pos[0]+ dir[0], self.pos[1] + dir[1])
        

    def out_of_bounds(self, next_pos):
        if next_pos[0] < 0: return True             # row under
        if next_pos[1] < 0: return True             # col under
        if next_pos[0] >= len(self.grid): return True    # row over
        if next_pos[1] >= len(self.grid[0]): return True # col over

    
    def turn(self):
        self.dir_ind = (self.dir_ind + 1) % 4



In [3]:
gsm = GridStateMachine(grid, start_pos,0, True)
gsm.run()
print(gsm.obs_list)
print("obs: ",len(gsm.obs_list))



FileNotFoundError: [Errno 2] No such file or directory: 'outputs'

In [ ]:
()

()